# Flood segmentation

Using a Kaggle dataset: [ETCi 2021 Competition on Flood Detection](https://www.kaggle.com/datasets/aninda/etci-2021-competition-on-flood-detection) (NASA contest)

* Objective: delineate open water flood areas as an effort to identify flood extent.
* Data: synthetic aperture radar (SAR) images.
* Performance: evaluated using the intersection over union (IOU) score.

## Setup

### Import necessary modules and do some basic setup.

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import os
import glob
import numpy as np

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


Matplotlib is building the font cache; this may take a moment.


## Dataset


